# Chapter 11 Scraping JavaScript

- JavaScript的主要功能：
    1. 收集追踪使用者的資訊
    1. 不重刷頁面就可以繳交表單
    1. 內嵌多媒體檔案
    1. 驅動整個網上遊戲
- 甚至是看起來很簡單的網頁，也常包含JavaScript。
- JavaScript出現在 stript 標記中。如：
  <img src = 'Chap11_01.png'>
  
## JavaScript簡介

- JavaScript是一個輕度有類型(weakly typed)的語言，語法類似C++或Java。
  <img src = 'chap11_02.png'>
- =======================我是分隔線===================================
  <img src = "Chap11_03.png">
- 常見JavaScript第三方函式庫：(直接使用Python來執行JS很慢，也很耗計算資源，能夠直接剖析而不執行，會很有用處)

    1. jQuery：
        1. 使用jQuery你會在JS碼中看到：<img src="Chap11_04.png">
        1. 因為jQuery會產生：只有執行JS碼後來動態產生的HTML內容，故要特別注意，不要只擷取到執行前的內容。(下面有一節仔細說明此點)
        1. jQuery常包括動畫、互動內容、包嵌之多媒體檔案等，可能對擷取產生困擾。
    1. Google Analytics：
        1. 網路上最常見的追踪使用者工具。
        1. 使用谷哥分析工具的網頁，會有：<img src="Chap11_05.png">
        1. 使用谷哥分析工具的網頁，在爬取時，不要讓網頁知道你在爬取它，要全然拋棄任何分析用的cookies或乾脆都不要用任何cookies。
    1. Google Maps:
        1. 在爬取有地點相關資訊的網頁時，了解谷哥地圖的運作讓你可以較容易取得經緯度、甚至地址資料。
        1. 谷哥地圖最常用來標示地點的JS碼：<img src="Chap11_06.png">
        1. 用Python來擷取一系列的經緯度是很容易的。
        1. 也可以用Google工具來從經緯度得到地址：https://developers.google.com/maps/documentation/javascript/examples/geocoding-reverse
        
## Ajax and Dynamic HTML

- 如果繳交表單或是下載資料而未刷新網頁，這個網頁很可能使用Ajax。
- Ajax全名是：Asynchronous JavaScript and XML，功能是：在不重新對網頁做要求(request)的情況下，傳送資訊給伺服器或從伺服器取得資料。
- Dynamic HTML (DHTML) 是 可以改變該頁的HTML元素的HTML碼/CSS語言：
    1. 某個紐只有在滑鼠移過去才出現
    1. 按紐才能改變的背景顏色
    1. 某個Ajax要求，可以驅使某些新內容載入
    1. 跟動畫、多媒體內容無關
- 在爬取時，遇到Ajax或/及DHTML，解決之道：
    1. 直接爬取JavaScript的內容
    1. 使用能執行JavaScript的Python套件，然後爬取你執行JS碼後看到的內容。
        

## 使用Selenium在Python中執行JavaScript

- Selenium本來是設計來做網頁測試用的工具。
- 現在，也常用在需要網頁真實呈現(像在瀏覽器中出現一樣)時。
- Selenium本身並不包含瀏覽器，故需要呼叫其他瀏覽器。
    1. firefox driver: https://github.com/mozilla/geckodriver/releases
    1. Chrome driver:http://chromedriver.chromium.org/
    1. 把下載的檔案解壓縮，然後複制進你放selenium的環境：C:\Users\Admin\Anaconda3\envs\scraping (這是我的環境)
- 在anaconda下安裝selenium：
    1. conda install -c conda-forge selenium
    1. conda install -c conda-forge/label/gcc7 selenium
    1. conda install -c conda-forge/label/cf201901 selenium
- 我們在以下網頁做測試：http://pythonscraping.com/pages/javascript/ajaxDemo.html。 這個網頁有一些內容，兩秒後，Ajax啟動，會跑出新的內容。

In [2]:
from selenium import webdriver
import time

browser = webdriver.Chrome()
browser.get('http://pythonscraping.com/pages/javascript/ajaxDemo.html')
time.sleep(3)
print(browser.find_element_by_id('content').text)
browser.close()

Here is some important text you want to retrieve!
A button to click!


### Some Explanation

- 上面是我們採用的書中的說法。
- 我看了之後，有個疑問：我怎麼知道要去找id=content呢？
- 如果我們依照之前的做法，在網頁上敲滑鼠右鍵，檢視網頁原始碼，並看不到這些東西。
- 這就是為什麼要使用網頁模擬器了：

In [7]:
from selenium import webdriver
import time

driver = webdriver.Chrome()

driver.get("http://pythonscraping.com/pages/javascript/ajaxDemo.html")
time.sleep(3)
print(driver.page_source)

<html><head>
<title>Some JavaScript-loaded content</title>
<script src="../js/jquery-2.1.1.min.js"></script>

</head>
<body>
<div id="content">Here is some important text you want to retrieve! <p></p><button id="loadedButton">A button to click!</button></div>

<script>
$.ajax({
    type: "GET",
    url: "loadedContent.php",
    success: function(response){

	setTimeout(function() {
	    $('#content').html(response);
	}, 2000);
    }
  });

function ajax_delay(str){
 setTimeout("str",2000);
}
</script>

</body></html>


In [8]:
# 等待三秒後，網頁模擬器會出現我們真正要的網頁
# 這個時候，我們可以用 .page_source()取得真正
# 的網頁的原始碼
# 這個時候，我們就可以用BeautifulSoup()來取得內容了！
# 當然，也可以用webdriver()內建的方便的功能來取得內容
# 比如 .find_element_by_id()等

from selenium import webdriver
import time

driver = webdriver.Chrome()

driver.get("http://pythonscraping.com/pages/javascript/ajaxDemo.html")
time.sleep(3) # driver.implicitly_wait()在 jupyter notebook
h = driver.page_source
bs = BeautifulSoup(h, 'html.parser')
print(bs.find('div').get_text())

Here is some important text you want to retrieve! A button to click!


### Selenium Selectors

- 在上面的例子中，我們使用：.find_element_by_id()來找 JS 碼中的content
- 以下做法可以達成相同目的：
    1. browser.find_element_by_css_selector('#content')
    1. browser.find_element_by_tag_name('div')<br><br>
- 如果要選多個成份，就在 element 上加s：
    1. browser.find_elements_by_css_selector('#content')
    1. browser.find_elements_by_tag_name('div')
    1. 以上回傳Python表列<br><br>
- 如果想用BeautifulSoup，則：
    pageSource = driver.page_source
    bs = BeautifulSoup(pageSource,'html.parser')
    print(bs.find(id='content').get_text())

- 上一格中，如果time.sleep()改為一秒，則會擷取到原來的、改變前的文本。
- 用時間來控制，可以有問題，因為，網頁的載入時間影響因素很多。
- 更有效率的做法是：持續檢查完全載入的網頁，看看某個特定的成份是否存在，而只有當該成份存在時，才回傳結果。
- 上面的測試網頁，Ajax執行後，會出現一個load按紐，故，以之來斷定真正想截取的內容是否已完全出現：

In [15]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

browser = webdriver.Chrome()
browser.get('http://pythonscraping.com/pages/javascript/ajaxDemo.html')

try:
    element = WebDriverWait(browser, 10).until(EC.presence_of_element_located((By.ID, 'loadedButton')))
    # <button id='loadedButton'>
finally:
    print(browser.find_element_by_id('content').text)
    browser.close()

Here is some important text you want to retrieve!
A button to click!


 - WebDriverWait()以及expected_conditions()是Selenium中稱為implicit wait的函式。
 - implicit wait vs. explicit wait:
     1. 前者等待某些DOM (domain object model: XML/HTML等的樹狀結構)串的狀態發生才會繼續
     1. 後者定義了硬性規定的時間 (如前例中的等三秒)
     1. 前者等待的DOM狀態由expected_conditions(縮寫為EC)定義
     1. expected_conditions可以是：
         1. 某種提示對話框出現
         1. 某個成份(如，一個文字框)被放進 selected 狀態。
         1. 頁面標題改變，或文本出現於網頁中/於某個特定成分中。
         1. 某成份現在在DOM中可以看得到了，或消失了。
 - expected_conditions中的成分，以locators來標示，並用By物件來使用。
     1. EC.presence_of_element_locate((By.ID, 'loadedButton')) # 用ID loadedButton來定義expected_conditions
     1. print(browser.find_element(By.ID, 'content').text) # 用 find_element及locators來建立selectors
     1. (browser.find_element_by_id('content').text) #與上功能相同
 - By物件使用下面的locators:
     1. ID: 用HTML ID來找
     1. CLASS_NAME: 用HTML的class屬性來找
     1. CSS_SELECTOR: 用class, id, tag這些標記來找，真正使用的方式為： #idName, .className, tagName
     1. LINK_TEXT: 找HTML中的 a 的文本，如：B.LINK_TEXT, "Next" (找叫Next的標籤 (label))
     1. PARTIAL_LINK_TEXT：同上，但只需部份文本即可
     1. NAME: 利用tag的name屬性來找tag，表單時特別好用
     1. TAG_NAME: 用tag的名字來找tag
     1. XPATH: 用 XPath 表達示(見下)來選擇對應成份

### XPath 語法

- XPath語法有四個主要概念：
    1. 根節點　vs. 非根節點
        1. /div (找在根節點的div)
        1. //div (找任何地點的div)
    1. 屬性
        1. //@href 選擇位於任何位置的href
        1. //a[@href='http://google.com'] 選擇所有指向google的連結
    1. 利用位置來選擇節點
        1. //a[3] 選檔案中第三個連結
        1. //table[last()] 選檔案中最後一個表格
        1. //a[position()>3]選檔案中前三個連結
    1. \* 任何字元或節點
        1. //table/tr/* 所有表格中，tr的所有子代 (用來選擇cell中th及td兩個標記很好用)
        1. //div[@\*] 選擇有任何屬性的div標記
- 更多相關資訊，請見：msdn.microsoft.com/en-us/enus/library/ms256471

- 一般而言，爬取JS資料並不需要在電腦螢幕上出現一個瀏覽器。
- 不過，出現瀏覽器有幾個好處：
    1. 除錯：如果程式碼有錯，能看到網頁會比較容易看到錯誤。
    1. 有些測試可能只能在某個特定瀏覽器上才能使用。
    1. 有些JS碼在不同瀏覽器有有稍微的不同，需要在瀏覽器上執行才看得出來。
- 除了Chrome、Firefox外，selenium還支援：
    1. safari_driver = webdriver.Safari()
    1. ie_driver = webdriver.Ie()
    1. 上面的驅動程式要上網蒐尋下載，並放在你的anaconda的scraping環境中

## 處理重新導向(redirect)

- 在：http://pythonscraping.com/pages/javasript/redirectDemo1.html 上示範
- 你可以用聰明的方法來偵測導新導向：注意某個在原先頁面載入時DOM中的元素，持續呼叫之，一直到selenium回傳：StaleElementReferenceException，因為，當頁面重新導向時，該元素就不會出現在頁面的DOM中。

In [2]:
from selenium import webdriver
import time
from selenium.webdriver.remote.webelement import WebElement
from selenium.common.exceptions import StaleElementReferenceException

def waitForLoad(driver):
    elem = driver.find_element_by_tag_name("html")
    count = 0
    while True:
        count += 1
        if count > 20:
            print('Timing out after 10 seconds and returning')
            return
        time.sleep(.5)
        try: 
            elem = driver.find_element_by_tag_name('html')
        except StaleElementReferenceException:
            return

driver = webdriver.Chrome()
driver.get('http://pythonscraping.com/pages/javascript/redirectDemo1.html')
waitForLoad(driver)
print(len(driver.page_source))

Timing out after 10 seconds and returning
119


- 也可以寫一個簡單的迴圈，檢查網頁的URL一直到其改變為止或是變成你在找的特定URL。
- 等待元素出現或消失是selenium常見的工作，可以使用之前用過的WebDriverWait：

In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

driver = webdriver.Firefox()
driver.get('http://pythonscraping.com/pages/javascript/redirectDemo1.html')
try:
    bodyElement = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, 
                                                 '//body[contains(text(), "This is the page you are looking for!")]')))
    print(bodyElement.text)
except TimeoutException:
    print('Did not find the element')
driver.close()

This is the page you are looking for!


## Final note

- 雖然大部份網頁包含JavaScript，但可能不影響我們爬取資料。
- 如果會影響，則可以使用像selenium等工具，來產生你已經會爬取的簡單網頁。